In [ ]:
"""
the code was done through spyder because we could not install the keras 
and tensorflow packages as well as many others in jupyter.
these were necessary when running the language model. 
"""

In [ ]:
# load in libraries
import itertools
import random
from random import sample
import re
import os
import csv
import string
import pickle
import nltk 
import collections
from nltk.corpus import stopwords
from pickle import load
from random import randint
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
import tensorflow 
import keras 
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, LSTM
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

In [ ]:
# string manipulation and creating the pandas dataframe
def txt(title):
    title = title[6:]
    title = title[:-4]
    title = re.sub("_", " ", title)
    title = re.sub("/", " ", title)
    return title

all_files = os.listdir("state_of_the_unions/")
correct_filenames = []
for item in sorted(all_files):
    correct_filenames.append("state_of_the_unions/" + item)
    
files = {}

for filename in correct_filenames:
    with open(filename, "r") as file:
        if filename in files:
            continue
        files[filename] = file.read()

df = pd.DataFrame(files.items(), columns=["pres_sotu", "speech"])
tokens = []

for row in df["speech"]:
    row = row.lower()
    sotu = re.split('[^a-zA-Z0-9]', row)
    clean_sotu = list(filter(None,sotu))
    tokens.append(clean_sotu)
  

president_name = []
year = []

for row in df['pres_sotu'].apply(txt):
    row = row.split(" ")
    president_name.append(row[-2])
    year.append(row[-1])
    
df["president"] = president_name
df["year"] = year
df['tokens'] = tokens

In [ ]:

"""
Start of Language Model Prediction
"""
# select trump 2017 speech
x = df.loc[df["year"] == "2017"]
tokens_2017 = x["tokens"].tolist()[0]
print("Total Tokens: " + str(len(tokens_2017)))
print("Unique Tokens: " + str(len(set(tokens_2017))))

# create specified number of sequences from a list of tokens
def create_sequences(list_of_tokens, len_of_seq):
    length = len_of_seq + 1
    sequences = list()
    for i in range(length, len(list_of_tokens)):
        seq = list_of_tokens[i-length:i]
        line = " ".join(seq)
        sequences.append(line)
    return sequences
print("Total Sequences: " + str(len(x)))

# save tokens to file one per line
def save_doc(lines, filename):
    text = "\n".join(lines)
    file = open(filename, "w")
    file.write(text)
    file.close()
    
trump_sequences = create_sequences(tokens_2017, 50)
trump = "trump_2017_sequences.txt"
save_doc(trump_sequences, trump)

# load sequences into memory
def load_doc(filename):
    file = open(filename, "r")
    text = file.read()
    file.close()
    return text

in_filename = "trump_2017_sequences.txt"
doc = load_doc(in_filename)
lines = doc.split("\n")

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = np.asarray(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
# define model

"""
vocabulary_size = size of vocabulary
num_of_layers = size of embedding vector space
len_of_sequence = length of sequences
mem_cells = LSTM number of memory cells
num_connect_neurons = number of neurons to connect with memory cells
"""
def model_lang(vocabulary_size, num_of_layers, len_of_sequence, mem_cells, num_connect_neurons):
    model = Sequential()
    model.add(Embedding(vocabulary_size, num_of_layers, input_length=len_of_sequence))
    model.add(LSTM(mem_cells, return_sequences=True))
    model.add(LSTM(mem_cells))
    model.add(Dense(num_connect_neurons, activation="relu"))
    model.add(Dense(vocabulary_size, activation="softmax"))
    return model

m1 = model_lang(vocab_size, 50, seq_length, 100, 100)
print(m1.summary())

m2 = model_lang(vocab_size, 300, seq_length, 300, 250)
print(m2.summary())

def compile_model(model):
    m=model
    m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return m

# compile model
m2_comp = compile_model(m2)

# fit model
m2_comp.fit(X, y, batch_size=100, epochs=100)
m2.save("m2.h5")

# save model to file
model.save("model.h5")
pickle.dump(tokenizer, open("tokenizer.pkl", "wb"))

In [ ]:
# predict a sequence of words from a given language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words):

        # make the text consist of integers instead of words
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        
        # pad and create sme length sequences
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    
        # predicted probabilities for words/sequences
        probs = model.predict_classes(encoded, verbose=0)
        
        # predicted words are keyed to index
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == probs:
                out_word = word
                break

    # append words to the result and return a string
    in_textext += ' ' + out_word
    result.append(out_word)
    return ' '.join(result)




In [ ]:
# this checks how similar one predicted text from the next sequence of words. 

cosines_2 = []
i_2 = 0
length = len(lines) - 3
while i_2 < length:
    for index in range(len(lines)):
        combined = list(set(lines[index] + lines[index+1] + lines[index+2]))
        train_text = " ".join(combined)
        test_text = lines[index+3]
        X_train = generate_seq(model, tokenizer, seq_length, train_text, len(test_text))
        sim = similar_to_which(X_train, test_text)[0][1]
        print(i_2)
        cosines_2.append(sim)
        print(np.mean(cosines_2))
        i_2+=1
        
        ##average cosine similarity is 0.34

In [ ]:
"""
Markov Chain Dictionary and Prediction
"""

president = df.loc[df["year"] == "2017"]
corpus = president["speech"].tolist()
corpus = " ".join(corpus)
corpus = corpus.replace("\n", " ")
corpus_lower = corpus.lower()
corpus = re.split('[^a-zA-Z0-9]', corpus)
clean = list(filter(None,corpus))


# creating bigrams from the split list in trump 2017 speech
def make_pairs(corp):
    for i in range(len(corp)-1):
        yield (corp[i], corp[i+1])

    
"""
Creates a dictionary with the keys as every single word and the values as the 
adjacent words to the particular word. 
"""
    
def markov_dictionary(data_list):

    # read in data_list called by the make_pairs function
    pairs = make_pairs(data_list)
    
    # initiate an empty dictionary for appending
    word_dictionary = {}
    for word1, word2 in pairs:
        if word1 in word_dictionary.keys():
            word_dictionary[word1].append(word2)
        else:
            word_dictionary[word1] = [word2]
    
    return(word_dictionary)
    
    
"""
Predicts from a start word a string of however long thats specific and returns a string of words. 
"""    

def markov_predict(word_dictionary, start_word, length):
    
    
    first_word = start_word
    #first_word = start_word.capitalize()
    #first_word = np.random.choice(data_list)
    
    #while first_word.islower():
    #    first_word = np.random.choice(word_dictionary)
    
    chain = [first_word]
    
    n = length
    
    for i in range(n):
        chain.append(np.random.choice(word_dictionary[chain[-1]]))
    
    output_string = " ".join(chain)
    
    return output_string

In [ ]:
"""
attempting to create some sort of validation for markov chain.
the model outputs the accuracy as it goes.
"""

def similar(a,b):
    return SequenceMatcher(None, a, b).ratio()


    
d = markov_dictionary(clean)

random_algo = random.sample(clean, 50)
    
train_text = lines[0]
test_text = lines[1]
X_model = generate_seq(model, tokenizer, seq_length, train_text, len(test_text))
mark = markov_predict(d, test_text.split(" ")[0],49)

print(similar_to_which(test_text, X_model))
print(similar_to_which(test_text, mark))
print(similar_to_which(" ".join(random_algo), test_text))


num_k_folds = 5

k_list = [clean[0:1019], clean[1019:2038], clean[2038:3057], clean[3057:4076], clean[4076:5095]]

sims_markov=[]
sims_random=[]
for ind in range(len(k_list)):
    a = k_list.copy()
    del a[ind]
    test_string = str(k_list[ind])
    train_list = list(itertools.chain.from_iterable(a))
    print(ind)

    
    m_dict = markov_dictionary(train_list)
    
    first_word = k_list[ind][0]
    
    i=0
    while first_word not in m_dict.keys():
        i+=1
        first_word=k_list[ind][i]
        
    
    train_predict = markov_predict(m_dict, first_word, 1018).lower()
    
    list_markov = train_predict.split(" ")
    
    counts_markov = Counter(list_markov)
    counts_test = Counter(test_string.lower().split(" "))
    
    common_markov = counts_markov.most_common()
    common_test = counts_test.most_common()
    
    #print(counts_markov)
    #print(counts_test)
    
    print(len(counts_markov))
    print(len(counts_test))
    
    count = 0
    if len(common_markov) > len(common_test):
        total = len(common_test)
        for k in range(total):
            if common_test[j][0] == common_markov[j][0]:
                count += 1
    else:
        total = len(common_markov)
        for j in range(total):
            if common_test[j][0] == common_markov[j][0]:
                count += 1
    print(count)
    
    
    
    similarity = similar(train_predict, test_string)
    
    ran = random.sample(train_list,1019)
    
    
    sims_random.append(similar(" ".join(ran), test_string))
    
    
    sims_markov.append(similarity)